## Hyperparameter search

Source: https://github.com/itdxer/neupy/blob/master/notebooks/Hyperparameter%20optimization%20for%20Neural%20Networks.ipynb
and http://neupy.com/2016/12/17/hyperparameter_optimization_for_neural_networks.html#id14

```python
Hyperparams to train:
1. lamda
2. number of hidden layers
3. number of hidden units
3. starter learning rate
5. use_sigmoid_h1
6. use_sigmoid_h2```

In [ ]:
import time
import os
import re
import sys
import numpy as np

sys.path = sys.path[1:]
sys.path.insert(0, os.path.join(os.getcwd(), "helper_scripts"))  # pretending we are running main.py in jupyter

import global_variables_final_for_git
reload(global_variables_final_for_git)
from global_variables_final_for_git import Global_Vars
from prep_for_model_for_git import Model_preparation

import hyperopt
from hyperopt import fmin
import HPO_helper
reload(HPO_helper)
from HPO_helper import uniform_int, loguniform_int, tpe_method, get_parameter_space_forHPO
import tensorflow_model_for_git
reload(tensorflow_model_for_git)
from tensorflow_model_for_git import Tensorflow_model

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

class Args(object):
    def __init__(self):
        self.gene = "TNNC2"
        self.distance = 150
        self.use_tad_info = True
        self.pcc_lowerlimit_to_filter_dhss = 0.2
        self.filter_tfs_by = "zscore" # or "pcc"
        self.lowerlimit_to_filter_tfs = 6
        self.take_this_many_top_dhss = 6  # all dhss/tfs will already be filtered by pcc(or zscore)
        self.take_this_many_top_tfs = 10
        self.init_wts_type = "corr"
        self.outputDir = "/Users/Dinesh/Dropbox/Github/predicting_gex_with_nn_git/Output_new/"+self.gene.upper()
        self.use_random_DHSs = False
        self.use_random_TFs = False 
        self.max_iter = 150
        self.plot_all = True

###test_idx = 17 # 2 to brain; 6 to ESC; 7 is epithelial; 8 is HSC and B; 12 is muscle; 5 is ES-derived
start_time = time.time()
args = Args()
gv = Global_Vars(args, args.outputDir)  # note this takes in new_output_dir as well in .py scripts

In [ ]:
def plot_input_data(gv):
    plt.figure(figsize=(12, 21))
    plt.subplot(1,3,1)
    sns.heatmap(gv.df_dhss.transpose(),yticklabels=True)
    plt.subplot(1,3,2)
    sns.heatmap(gv.df_tfs.transpose(), yticklabels=False)
    plt.subplot(1,3,3)
    sns.heatmap(gv.goi.to_frame(), yticklabels=False)
    plt.savefig(os.path.join(gv.outputDir, "input_feats_and_goi.pdf"))
plot_input_data(gv)

In [ ]:
'''Run HPO on differen train/test splits'''
mp = Model_preparation(gv)
start_time = time.time()

'''After updates - what i had before'''

tm.get_importance_scores(updates)

to_log, plot_title = tm.get_log_info_to_save_after_retraining(trainY, updates, title_prefix=title_prefix)  # title prefix only has mode, test group, and testX.shape infos
print(to_log)  # logger.info(to_log)
plot_title = "{};{}".format(gv.gene_ofInterest, plot_title)
tm.plot_performance_after_retraining(amode, gv, updates, trainY, plot_title=plot_title)

del tm, trials, best_params, index
del to_log, title_info, title_prefix, title_error_msg, title_suffix, plot_title
del wts, layer_sizes, lamda, trainX, trainY, b1, g1, b2, g2, updates

In [ ]:
import collections as col
import copy
dict_ims = col.OrderedDict()  # {<feat_index>: [rmse]}  # typically this will only have one or few elements since running for lots of test groups take a long time

for amode in ["joint"]: # ["joint"]: #, "tfs", "dhss"]:
    for test_idx in range(1,2):
        if (test_idx == 4):
            continue
        tm = Tensorflow_model(gv, mp, test_eid_group_index=test_idx, mode=amode)
        trials = hyperopt.Trials()
        best_params = hyperopt.fmin(tm.train_tensorflow_nn, trials=trials,
                                    space=get_parameter_space_forHPO(tm.trainX),
                                    algo=tpe_method, max_evals=8)

        index = np.argmin(trials.losses())
        to_log = tm.get_log_into_to_save(index, trials, best_params, mode=amode)
        print(to_log) # logger.info(to_log)

        title_info = re.split(";best_params", to_log)[0]  # "title" refers to plot_title
        title_prefix, title_error_msg, title_suffix = re.split(";median_pc_error|;PCC", title_info)  # note title_prefix already has the "mode" info
        title_info = title_prefix + "\nmed_pc_err" + title_error_msg + "\nPCC" + title_suffix
        plot_title = "{};{}".format(gv.gene_ofInterest, title_info)
        tm.plot_scatter_performance(amode, index, trials, gv, plot_title=plot_title)

        # Now retrain using the validation set
        starter_lr, use_sigmoid_h1, use_sigmoid_h2, use_sigmoid_yhat, wts, layer_sizes, lamda, trainX, trainY, b1, g1, b2, g2 = tm.get_params_from_best_trial(index, trials, best_params)  # nn_updates is the new dict, not the one in tf_model class
        updates = tm.retrain_tensorflow_nn(starter_lr, use_sigmoid_h1, use_sigmoid_h2, use_sigmoid_yhat, wts, layer_sizes, lamda, trainX, trainY, b1, g1, b2, g2)
        
        # collect the rmses
        X_ori = np.concatenate((trainX, tm.testX), axis=0)  # trainX = tm.trainX + tm.valX
        Y_ori = np.concatenate((trainY, tm.testY), axis=0)  # this should not be imputed
        r_ori = tm.get_rmse(updates, use_sigmoid_h1, use_sigmoid_h2, use_sigmoid_yhat, layer_sizes, lamda, X_=X_ori, Y_ori=Y_ori)
        for i in range(0, X_ori.shape[1]):  # for all features
            X_ = copy.deepcopy(X_ori)
            X_[:,i] = 0  # mutate the feat column
            r_ = tm.get_rmse(updates, use_sigmoid_h1, use_sigmoid_h2, use_sigmoid_yhat, layer_sizes, lamda, X_=X_, Y_ori=Y_ori)  # only X is mutated 
            if (i in dict_ims.keys()):
                dict_ims[i].append(abs(r_ori - r_))
            else:
                dict_ims[i] = [abs(r_ori - r_)]
            del X_
            
            # if feature index is ___: do perturbation in silico experiments
        write_rmses(dict_ims)
    
        to_log, plot_title = tm.get_log_info_to_save_after_retraining(trainY, updates, title_prefix=title_prefix)  # title prefix only has mode, test group, and testX.shape infos
        print(to_log)  # logger.info(to_log)
        plot_title = "{};{}".format(gv.gene_ofInterest, plot_title)
        tm.plot_performance_after_retraining(amode, gv, updates, trainY, plot_title=plot_title)

        '''del tm, trials, best_params, index
        del to_log, title_info, title_prefix, title_error_msg, title_suffix, plot_title
        del wts, layer_sizes, lamda, trainX, trainY, b1, g1, b2, g2, updates'''
    
        print("done with mode:", amode)


In [ ]:
def write_rmses(adict_ims):
    '''Write the RMSEs (importance scores) corresponding to feature mutations in a file. 
    Arguments:
    - adict_ims is the importance score dict, with a list of rmses for each feature index keys.
    '''
    handleIn = open(os.path.join(gv.outputDir, "feat_importance_scores.txt"), "aw")
    for k,v in adict_ims.items():
        for av in v: # for each rmse value corresponding to the feature
            handleIn.write("{}:{}".format(k,av))
    handleIn.close()

In [ ]:
adict = dict_ims # from 8 to 11 inclusive (0 based indexing)
for k,v in adict.items():
    print(k,len(v), v)

In [ ]:
plt.plot(adict.values())

## Predicting the effect of increasing predictor signal to gene expression

In [ ]:
yhat = tm.get_yhat(updates, use_sigmoid_h1, use_sigmoid_h2, use_sigmoid_yhat, layer_sizes, lamda, X_=X_ori, Y_ori=Y_ori)  

In [ ]:
plt.scatter(yhat[0].flatten(), Y_ori.flatten())
plt.xlim(0,1)
plt.ylim(0,1)

In [ ]:
sns.set(font_scale=1.5)
plt.figure(figsize=(5,5))
yhat = tm.get_yhat(updates, use_sigmoid_h1, use_sigmoid_h2, use_sigmoid_yhat, layer_sizes, lamda, X_=X_ori, Y_ori=Y_ori)
plt.scatter(Y_ori.flatten(), yhat[0].flatten(), alpha=0.3)

X_ = copy.deepcopy(tm.valX)
yval_hat = tm.get_yhat(updates, use_sigmoid_h1, use_sigmoid_h2, use_sigmoid_yhat, layer_sizes, lamda, X_=X_, Y_ori=tm.valY)
sns.regplot(tm.valY.flatten(), yval_hat[0].flatten(), color="salmon")

plt.xlim(0,1)
plt.ylim(0,1)
plt.xlabel("Real expression")
plt.ylabel("Predicted expression")

In [ ]:
zip(tm.val_goi, yval_hat[0].flatten())

In [ ]:
#feat_index = 3 # feature index in X matrices
max_fold = 5  # max fold increase

index_hsmm = np.where(tm.valY.flatten() == tm.val_goi["E120_MUS.HSMM_ENCODE2012"])[0][0]
index_hsmmt = np.where(tm.valY.flatten() == tm.val_goi["E121_MUS.HSMMT_ENCODE2012"])[0][0]
index_brain = np.where(tm.valY.flatten() == tm.val_goi["E125_BRN.NHA_ENCODE2012"])[0][0]
index_blood = np.where(tm.valY.flatten() == tm.val_goi["E124_BLD.CD14.MONO_ENCODE2012"])[0][0]

for feat_index in [6,7,8]:
    expn_in_hsmm = []
    expn_in_hsmmt = []
    expn_in_brain = []
    expn_in_blood = []

    for i in range(1, max_fold): # from 1x to 5x
        X_ = copy.deepcopy(tm.valX)
        X_[:,feat_index] *= i
        yval_hat = tm.get_yhat(updates, use_sigmoid_h1, use_sigmoid_h2, use_sigmoid_yhat, layer_sizes, lamda, X_=X_, Y_ori=tm.valY)
        expn_in_hsmm.append(yval_hat[0].flatten()[index_hsmm])
        expn_in_hsmmt.append(yval_hat[0].flatten()[index_hsmmt])
        expn_in_brain.append(yval_hat[0].flatten()[index_brain])
        expn_in_blood.append(yval_hat[0].flatten()[index_blood])
        print(expn_in_hsmm) 
        print(expn_in_hsmmt) 

    '''Plot the expn in cell line tracked'''
    only_hsmmt = True
    plt.plot(range(1,max_fold), expn_in_hsmmt, color="red")
    plt.axhline(y=tm.val_goi[index_hsmmt], color="red", linestyle="--", label="hsmmt")

    if (not only_hsmmt):
        plt.plot(range(1,max_fold), expn_in_hsmm, color="orange")
        plt.plot(range(1,max_fold), expn_in_brain, color="blue")
        plt.plot(range(1,max_fold), expn_in_blood, color="green")
        plt.axhline(y=tm.val_goi[index_hsmm], color="orange", linestyle="--", label="hsmm")
        plt.axhline(y=tm.val_goi[index_brain], color="blue", linestyle="--", label="brain")
        plt.axhline(y=tm.val_goi[index_blood], color="green", linestyle="--", label="blood")

    plt.legend()
    plt.xlabel("Fold increase")
    plt.ylabel("Normalized predicted expression")

    if (only_hsmmt):
        plt.savefig(gv.outputDir+"/zerobased_featIndex{}_overexpn_for_tnnc2_in_hsmmtOnly.pdf".format(feat_index))
    else:
        plt.savefig(gv.outputDir+"/zerobased_featIndex{}_overexpn_for_tnnc2.pdf".format(feat_index))
    plt.close()

#saving upto idx=7 (0-based) 
adict_to7 = dict_ims
for k,v in adict_to7.items():
    print(k,len(v), v)

###adict = {"a":[1,2,3], 1:[11,21,31]}
df = pd.DataFrame.from_dict(adict, orient='columns', dtype=None)
df_m = pd.melt(df, id_vars=["index"], value_vars=df.columns.tolist())  
#note: first 2 elements in columns list are "level_0" and index" now 
df_m.columns = ["index", "Feature", "Importance Score"]
df_m = df_m[["Feature", "Importance Score"]]

sns.set(font_scale=1.5)
sns.boxplot(x="Feature", y="Importance Score", data=df_m)
plt.xticks(rotation=85)
plt.tight_layout()

In [ ]:
# - EOF - 

```python
# SINCE LAST COMMIT

'''Run HPO on differen train/test splits'''
tm = Tensorflow_model(gv, mp, test_eid_group_index=test_idx, mode = "joint")  # mode should be one of "dhss", "tfs", "joint"

plt.figure(figsize=(10,10))
plt.subplot(2, 2, 1)
sns.heatmap(tm.trainX, vmin=0, vmax=1)
plt.title('trainX')
plt.subplot(2, 2, 2)
sns.heatmap(tm.trainY, vmin=0, vmax=1)
plt.title('trainY')

plt.subplot(2, 2, 3)
sns.heatmap(tm.testX, vmin=0, vmax=1)
plt.title('testX')

plt.subplot(2, 2, 4)
sns.heatmap(tm.testY, vmin=0, vmax=1)
plt.title('testY')
plt.suptitle("{}, test_idx:{}".format(gv.gene_ofInterest, test_idx))

'''Now train and search for HPO'''
trials = hyperopt.Trials()
best_params = hyperopt.fmin(tm.train_tensorflow_nn, trials=trials,
                            space=get_parameter_space_forHPO(tm.trainX),
                            algo=tpe_method, max_evals=10)
index = np.argmin(trials.losses())
to_log = tm.get_log_into_to_save(index, trials, best_params)
plot_title = re.split(";best_params", to_log)[0]
plot_title_prefix, plot_title_error_msg, plot_title_suffix = re.split(";median_pc_error|;PCC", plot_title)
plot_title = plot_title_prefix + "\nmed_pc_err" + plot_title_error_msg + "\nPCC" + plot_title_suffix
tm.plot_scatter_performance(trials, gv, gv.gene_ofInterest + ";" + plot_title)

print("Total time taken: {}".format(time.time() - start_time))

index = np.argmin(trials.losses())
plt.plot(trials.results[index]["train_loss"], color="salmon")
plt.plot(trials.results[index]["val_loss"], color="mediumvioletred")

index = np.argmin(trials.losses())
wts, layer_sizes, lamda, trainX, trainY, nn_updates, b1, g1, b2, g2 = tm.get_params_from_best_trial(index, trials, best_params)
updates = tm.retrain_tensorflow_nn(wts, layer_sizes, lamda, trainX, trainY, nn_updates, b1, g1, b2, g2)
tm.plot_performance_after_retraining(gv, updates, trainY, title_prefix=plot_title_prefix)  # need to pass in the title_prefix
```

In [ ]:
# plotting six1 vs tnnc2 gene expn
X_ori = np.concatenate((trainX, tm.testX), axis=0)  # trainX = tm.trainX + tm.valX
Y_ori = np.concatenate((trainY, tm.testY), axis=0)  # this should not be imputed

In [ ]:
plt.scatter(X_ori[:,6], Y_ori.flatten())
plt.title("{}".format(np.corrcoef(np.array(X_ori[:,6]), np.array(Y_ori.flatten()))))
plt.xlim(0,1)

In [ ]:
plt.scatter(X_ori[:,7], Y_ori.flatten())
plt.title("{}".format(np.corrcoef(np.array(X_ori[:,7]), np.array(Y_ori.flatten()))))
plt.xlim(0,1)

In [ ]:
plt.scatter(X_ori[:,8], Y_ori.flatten())
plt.title("{}".format(np.corrcoef(np.array(X_ori[:,8]), np.array(Y_ori.flatten()))))
plt.xlim(0,1)

In [ ]:
plt.scatter(X_ori[:,3], Y_ori.flatten())
plt.title("{}".format(np.corrcoef(np.array(X_ori[:,3]), np.array(Y_ori.flatten()))))
plt.xlim(0,1)

In [ ]:
plt.scatter(X_ori[:,12], Y_ori.flatten())
plt.title("{}".format(np.corrcoef(np.array(X_ori[:,12]), np.array(Y_ori.flatten()))))
plt.xlim(0,1)

In [ ]:
plt.scatter(X_ori[:,10], Y_ori.flatten())
plt.title("{}".format(np.corrcoef(np.array(X_ori[:,10]), np.array(Y_ori.flatten()))))
plt.xlim(0,1)